In [ ]:
%load_ext autoreload
%load_ext aymurai.devtools.magic
%autoreload 2

In [ ]:
import locale

from sklearn.model_selection import train_test_split

from aymurai.spacy.display import DocRender
from aymurai.pipeline import AymurAIPipeline
from aymurai.datasets.ar_juz_pcyf_10 import ArgentinaJuzgadoPCyF10Dataset

locale.setlocale(locale.LC_ALL, "es_AR.UTF-8")

render = DocRender()

In [ ]:
docs = ArgentinaJuzgadoPCyF10Dataset('private-docs', use_cache=False)
docs = filter(lambda x: 'admisibilidad_prueba' not in x['metadata']['objeto_de_la_resolucion'], docs)
docs = list(docs)
print('docs', len(docs))

In [ ]:
import aymurai.spacy.components.loader
from aymurai.spacy.ruler import SpacyRulerPipeline
from aymurai.text.extraction import FulltextExtract
from aymurai.utils.entities import FilterEntity
from aymurai.text.normalize import JunkCleaner, TextNormalize
from aymurai.models.dummy.decision import DummyExtractorDecision
from aymurai.models.dummy.n_expte_eje import DummyExtractorExpediente
from aymurai.models.dummy.affiliation import DummyExtractorAffiliation
from aymurai.spacy.components.es_ar.keywords import SpacyRulerKeywords
from aymurai.spacy.rulers.section_parser import AymuraiRulerSectionParser
from aymurai.models.dummy.art_infringido import DummyExtractorArtInfringido
from aymurai.models.dummy.tipo_resolucion import DummyExtractorTipoResolucion
from aymurai.models.dummy.fecha_resolucion import DummyExtractorFechaResolucion
from aymurai.spacy.components.es_ar.art_infringido import SpacyRulerArtInfringido
from aymurai.models.dummy.hora_inicio_cierre import DummyExtractorHoraInicioCierre

config = {
    "preprocess": [
        (
            FulltextExtract,
            {
                "errors": "ignore",
                "use_cache": True,
            },
        ),
        (TextNormalize, {}),
        (
            JunkCleaner,
            {
                "patterns": [
                    "Juzgado PCyF N* 10 - Tacuarí 138, 7* Piso - juzcyf10ejusbaires.gob.ar - 4014-6821/20 - Gipcyf10",
                ]
            },
        ),
        (
            SpacyRulerPipeline,
            {
                "base": "es",
                "steps": [
                    (
                        "enhanced_regex_ruler",
                        {
                            "patterns": {
                                "DATE": [
                                    "%-d/%-m/%Y",
                                    "%-d/%-m/%y",
                                    "%d/%m/%Y",
                                    "%d/%m/%y",
                                    "(?i)%-d de %B del? %Y",
                                ],
                                "TIME": [
                                    "%H(\.|:)%M",
                                    "%-H(.|:)%M (?i)horas",
                                    "%-H.%M h(rs|r|s)\.?",
                                ],
                                "N_EXPTE_EJE": [
                                    r"(?i)causa\s*(n.)?\s*\d+/%Y(-\d)?",
                                    r"(?i)causa\s*(n.)?\s*\d+/%y(-\d)?",
                                    r"(?i)caso\s*(n.)?\s*\d+/%Y(-\d)?",
                                    r"(?i)caso\s*(n.)?\s*\d+/%y(-\d)?",
                                    r"EXP:\s*\d+/%Y(-\d)?",
                                    r"EXP:\s*\d+/%y(-\d)?",
                                    r"IPP?\s*\d+/%Y(-\d)?",
                                    r"IPP?\s*\d+/%y(-\d)?",
                                ],
                                "CONTEXTO_VIOLENCIA": [
                                    r'(?i)contexto de violencia (de )?\w+'
                                ]
                            },
                        },
                    ),
                    # frases violencia
                    ("aymurai_violence_quotes_ruler", {}),
                    # Names Ruler
                    # note that requires download facebook database
                    # FIXME: tarfile extraction is broken
                    # ("name_lookup_ruler", {"country_codes": ["AR"]}),
                    # ("join_consecutive_names", {}),
                ],
            },
        ),
        (
            AymuraiRulerSectionParser,
            {
                "base": "es",
                "breakpoints": {
                    "SECTION:DEVELOPMENT": [
                        "DESARROLLO",
                    ],
                    "SECTION:BACKGROUND": [
                        "ANTECEDENTES",
                        "ANTECEDENTES Y ARGUMENTOS",
                    ],
                    "SECTION:ARGUMENTS": [
                        "ARGUMENTOS",
                        "ANTECEDENTES Y ARGUMENTOS",
                        "CONSIDERO",
                    ],
                    "SECTION:DECISION": [
                        "DECID[EO]",
                        "RESUELV[EO]",
                        "DELIVER[AO]"
                    ],
                    "KEYWORDS": [
                        "PALABRAS[_\s]+CLAVE[\w\d\s_:]+",
                    ],
                },
            },
        ),
        (SpacyRulerArtInfringido, {}),
        (SpacyRulerKeywords, {}),
    ],
    "models": [
        (DummyExtractorFechaResolucion, {}),
        (DummyExtractorHoraInicioCierre, {}),
        (DummyExtractorExpediente, {}),
        (DummyExtractorDecision, {}),
        (DummyExtractorTipoResolucion, {}),
        (DummyExtractorAffiliation, {}),
    ],
    "postprocess": [
        (FilterEntity, {'entities': 'PER'})
    ],
    "multiprocessing": {},
    # "use_cache": True,
    "use_cache": True,
    # 'log_level': 'debug'
}

pipeline = AymurAIPipeline(config)

In [ ]:
preprocessed = pipeline.preprocess([docs[i] for i in range(10)])
# preprocessed = pipeline.preprocess([docs[i] for i in range(len(docs))])
result = pipeline.predict(preprocessed)
result = pipeline.postprocess(result)

In [ ]:
import json

import pandas as pd

registry = result[3]
# registry = preprocessed[85]
# registry = example[1]
print(registry['path'])
metadata = {
    k: v for k, v in registry["metadata"].items() if type(v) not in [dict, list]
}
print(json.dumps(metadata, indent=4))

print("\n-------\n")
if 'predictions' in registry:
    pred_cats = {
        k: v
        for k, v in registry["predictions"]["doc-cats"].items()
        if type(v) not in [dict, list]
    }
    display(pd.DataFrame(pred_cats, index=pd.Index([0])).T)


render = DocRender(config={"colors": {
    'AYMURAI_VIOLENCE_QUOTE': '#f95d6a',
    'CONTEXTO_VIOLENCIA': '#d45087',
    'FIRMA': '#007bb3',
    # 'FECHA_RESOLUCION': , 
    'ART_INFRINGIDO': '#665191',
    'CONDUCTA': '#665191',
    'N_EXPTE_EJE': '#2f4b7c',
}})
render(registry)
